In [ ]:
def calculate_mean_std(df_names, dfs):
    perf_train, perf_fair = [], []

    for df_name, df in zip(df_names, dfs):
        df_train = df[df['data'] == 'train_cont_ord_cat'].drop(columns=['data'])
        df_fair = df[df['data'] == 'fair_cont_ord_cat'].drop(columns=['data'])
        
        df_train_mean = df_train.mean()
        df_train_std = df_train.std()
        #df_train_median = df_train.median()
        df_train_mean['dataset'] = df_name
        df_train_std['dataset'] = df_name
        #df_train_median['dataset'] = df_name
        df_train_mean['type'] = 'mean'
        #df_train_median['type'] = 'median'
        df_train_std['type'] = 'std'
        perf_train.append(df_train_mean.to_frame().T)
        #perf_train.append(df_train_median.to_frame().T)
        perf_train.append(df_train_std.to_frame().T)
        
        df_fair_mean = df_fair.mean()
        #df_fair_median = df_fair.median()
        df_fair_std = df_fair.std()
        df_fair_mean['dataset'] = df_name
        #df_fair_median['dataset'] = df_name
        df_fair_std['dataset'] = df_name
        df_fair_mean['type'] = 'mean'
        #df_fair_median['type'] = 'median'
        df_fair_std['type'] = 'std'
        perf_fair.append(df_fair_mean.to_frame().T)
        #perf_fair.append(df_fair_median.to_frame().T)
        perf_fair.append(df_fair_std.to_frame().T)
    
    perf_train = pd.concat(perf_train).reset_index(drop=True)
    perf_fair = pd.concat(perf_fair).reset_index(drop=True)
    return perf_train, perf_fair

In [ ]:
def aggregate_runs(dfs, drop_columns = None):
    df = pd.concat(dfs).reset_index(drop=True)
    df.drop(columns=['iteration'], inplace=True)
    if drop_columns is not None:
        df.drop(columns=drop_columns, inplace=True)
    df_num_cols = [c for c in df.columns if c not in ['classifier', 'algorithm']]
    df.loc[:, df_num_cols] = df.loc[:, df_num_cols].abs()
    df = df.groupby(['classifier', 'algorithm']).agg(['mean', 'std'])
    return df

In [ ]:
def print_to_latex(dfs_f, dfs_p, drop_columns_f=None, drop_columns_p=None):
    df_f = aggregate_runs(dfs_f, drop_columns=drop_columns_f)
    df_p = aggregate_runs(dfs_p, drop_columns=drop_columns_p)
    df = df_f.join(df_p)
    print(df.to_latex(index=True,
                  float_format="{:.4f}".format,
))  

In [ ]:
def aggregate_runs_c(dfs, drop_columns=None, fair=True):
    """
    Aggregates runs by concatenating DataFrames, dropping specified columns,
    taking the absolute value of numerical columns, and calculating mean and
    standard deviation grouped by 'classifier' and 'algorithm'.
    """
    df = pd.concat(dfs).reset_index(drop=True)
    df.drop(columns=['iteration'], inplace=True)
    if drop_columns is not None:
        df.drop(columns=drop_columns, inplace=True)
    df['classifier'].replace({'decision_tree': 'Decision Tree', 'logistic_regression': 'Logistic Regression', 'mlp': 'MLP'}, inplace=True)
    df['algorithm'].replace({'fair_rbu': 'Fair-RBU', 'fair_rbh': 'Fair-RBH', 'fos': 'FOS', 'fawos': 'FAWOS', 'hfos': 'HFOS'}, inplace=True)
    if fair:
        df.rename(columns={'statistical_parity': 'SPD', 'average_absolute_odds': 'AAO', 'equal_opportunity': 'EOD'}, inplace=True)
        features = ['SPD', 'AAO', 'EOD']
    else:
        df.rename(columns={'accuracy': 'Accuracy', 'f1': 'F1', 'gmean': 'G-mean'}, inplace=True)
        features = ['Accuracy', 'F1', 'G-mean']
    df_num_cols = [c for c in df.columns if c not in ['classifier', 'algorithm']]
    df.loc[:, df_num_cols] = df.loc[:, df_num_cols].abs()
    
    grouped = df.groupby(['classifier', 'algorithm']).agg(['mean', 'std'])
    
    # Combine mean and std into one cell
    combined = grouped.copy()
    for col in df_num_cols:
        combined[(col, 'mean')] = grouped[(col, 'mean')].map('{:.4f}'.format) + ' (\\textpm' + grouped[(col, 'std')].map('{:.3f}'.format) + ')'
        combined.drop((col, 'std'), axis=1, inplace=True)
    
    return combined

def print_to_latex_c(dfs_f, dfs_p, drop_columns_f=None, drop_columns_p=None):
    """
    Aggregates and joins two sets of DataFrames, and prints the resulting
    DataFrame to LaTeX format.
    """
    
    df_f = aggregate_runs_c(dfs_f, drop_columns=drop_columns_f, fair=True)
    
    df_p = aggregate_runs_c(dfs_p, drop_columns=drop_columns_p, fair=False)
    df = df_f.join(df_p)
    
    print(df.to_latex(index=True, escape=False))

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

def aggregate_runs_no_mean(dfs, fair, dataset_name, bin, save_path='../figures'):
    df = pd.concat(dfs).reset_index(drop=True)
    df_num_cols = [c for c in df.columns if c not in ['classifier', 'algorithm']]
    df.loc[:, df_num_cols] = df.loc[:, df_num_cols].abs()
    df['classifier'].replace({'decision_tree': 'Decision Tree', 'logistic_regression': 'Logistic Regression', 'mlp': 'MLP'}, inplace=True)
    df['algorithm'].replace({'fair_rbu': 'Fair-RBU', 'fair_rbh': 'Fair-RBH', 'fos': 'FOS', 'fawos': 'FAWOS', 'hfos': 'HFOS'}, inplace=True)
    if fair:
        df.rename(columns={'statistical_parity': 'SPD', 'average_absolute_odds': 'AAO', 'equal_opportunity': 'EOD'}, inplace=True)
        features = ['SPD', 'AAO', 'EOD']
    else:
        df.rename(columns={'accuracy': 'Accuracy', 'f1': 'F1', 'gmean': 'G-mean'}, inplace=True)
        features = ['Accuracy', 'F1', 'G-mean']
    df = df.melt(id_vars=['classifier', 'algorithm'], value_vars=features)
    df.rename(columns={'variable': 'metric'}, inplace=True)
    sns.set(font_scale=0.75) 
    g = sns.FacetGrid(df, row="classifier", col='metric', row_order=['Decision Tree', 'Logistic Regression', 'MLP'], col_order=features, height=3, aspect=1.25)
    g.map_dataframe(sns.boxplot, x='algorithm', y='value', hue='algorithm', palette='colorblind', order=['-', 'FAWOS', 'FOS', 'Fair-RBH', 'Fair-RBU', 'HFOS'])
    plt.tight_layout()
    plt.savefig(f'{save_path}/{dataset_name}_{bin}_results_{fair}.pdf')
    plt.show()

# Cont ord cat

In [ ]:
import os
import pandas as pd

dfs_fairness = []
dfs_performance = []
dataset_names = ['bank']#, 'german', 'adult', 'bank']
#dataset_names = ['adult_' + i[:-4] for i in os.listdir('../data/adult_census/sampled_all/new') if '_balanced_g_' not in i]
#dataset_names = [d for d in dataset_names if '_balanced_g' not in d]
classifier_names = ['decision_tree', 'mlp', 'logistic_regression']
algorithm_names = ['fair_rbh', 'fair_rbu', 'fos', 'hfos', 'fawos']# 'fawos_hybrid']
#algorithm_names = ['fos']
folder_names = ['2024-07-14', '2024-07-14', '2024-06-29', '2024-06-29', '2024-06-29', '2024-08-24']
#folder_names = ['all_cont']
main_path = '../results'

print(dataset_names)

for dataset_name in dataset_names:
    df_data_fair = []
    df_data_perf = []
    for classifier_name in classifier_names:
        for folder_name, algorithm_name in zip(folder_names, algorithm_names):
            for i in range(0, 10):
                if os.path.exists(os.path.join(main_path, f'{algorithm_name}_{dataset_name}_{classifier_name}', folder_name, f'fairness_{i}.csv')):
                    fair_path = os.path.join(main_path, f'{algorithm_name}_{dataset_name}_{classifier_name}', folder_name, f'fairness_{i}.csv')
                    perf_path = os.path.join(main_path, f'{algorithm_name}_{dataset_name}_{classifier_name}', folder_name, f'performance_{i}.csv')
                    df_fair = pd.read_csv(fair_path)
                    df_performance = pd.read_csv(perf_path)
                    df_fair['classifier'] = [classifier_name] * len(df_fair)
                    df_fair['algorithm'] = [f'{algorithm_name}' if 'fair' in x else '-' for x in df_fair['data']]
                    df_fair['iteration'] = [i] * len(df_fair)
                    # df_fair.drop(columns=['data', 'average_odds', 'average_absolute_odds'], inplace=True)
                    df_fair.drop(columns=['data'], inplace=True)
                    df_performance['classifier'] = [classifier_name] * len(df_performance)
                    df_performance['algorithm'] = [f'{algorithm_name}' if 'fair' in x else '-' for x in df_performance['data']]
                    df_performance['iteration'] = [i] * len(df_performance)
                    df_performance.drop(columns=['data'], inplace=True)
                    df_data_fair.append(df_fair)
                    df_data_perf.append(df_performance)
    dfs_fairness.append(df_data_fair)
    dfs_performance.append(df_data_perf)
                

In [ ]:
for dataset_name, dfs in zip(dataset_names, dfs_fairness):
    df = aggregate_runs(dfs, drop_columns=['accuracy', 'disparate_impact', 'average_odds', ])
    print(dataset_name)
    display(df)
    print('----------------------------------------------------------------')

In [ ]:
for dataset_name, dfs_f, dfs_p in zip(dataset_names, dfs_fairness, dfs_performance):
    df = aggregate_runs(dfs_p, drop_columns=['balanced_accuracy',])
    print(dataset_name)
    display(df)
    print('----------------------------------------------------------------')

In [ ]:
for dataset_name, dfs_f, dfs_p in zip(dataset_names, dfs_fairness, dfs_performance):
    print(dataset_name)
    print_to_latex_c(dfs_f, dfs_p, drop_columns_f=['accuracy', 'disparate_impact', 'average_odds', 'adapted_disparate_impact'], drop_columns_p=['balanced_accuracy',])
    print('----------------------------------------------------------------')

In [ ]:

for dataset_name, dfs in zip(dataset_names, dfs_performance):
    print(dataset_name)
    print_to_latex(dfs)
    print('----------------------------------------------------------------')

In [ ]:
for dataset_name, dfs in zip(dataset_names, dfs_fairness):
    print(dataset_name)
    aggregate_runs_no_mean(dfs, True, dataset_name, 'bin')

In [ ]:
for dataset_name, dfs in zip(dataset_names, dfs_performance):
    print(dataset_name)
    aggregate_runs_no_mean(dfs, False, dataset_name, 'bin')

In [ ]:
def aggregate_runs_no_mean(dfs_f, dfs_p, dataset_name, bin, save_path='../figures'):
    
    df = pd.concat(dfs_f).reset_index(drop=True).drop(columns=['accuracy'])
    df1 = pd.concat(dfs_p).reset_index(drop=True)
    df = df.set_index(['classifier', 'algorithm', 'iteration']).join(df1.set_index(['classifier', 'algorithm', 'iteration']))
    df = df.reset_index(drop=False)
    df_num_cols = [c for c in df.columns if c not in ['classifier', 'algorithm']]
    df.loc[:, df_num_cols] = df.loc[:, df_num_cols].abs()
    df['classifier'].replace({'decision_tree': 'Decision Tree', 'logistic_regression': 'Logistic Regression', 'mlp': 'MLP'}, inplace=True)
    df['algorithm'].replace({'fair_rbu': 'Fair-RBU', 'fair_rbh': 'Fair-RBH', 'fos': 'FOS', 'fawos': 'FAWOS', 'hfos': 'HFOS'}, inplace=True)
    
    df.rename(columns={'statistical_parity': 'SPD', 'average_absolute_odds': 'AAO', 'equal_opportunity': 'EOD'}, inplace=True)
    df.rename(columns={'accuracy': 'Accuracy', 'f1': 'F1', 'gmean': 'G-mean'}, inplace=True)
    features = ['SPD', 'EOD','AAO', 'Accuracy', 'F1', 'G-mean']
    df = df.melt(id_vars=['classifier', 'algorithm'], value_vars=features)
    df.rename(columns={'variable': 'metric'}, inplace=True)
    sns.set(font_scale=0.75, style='whitegrid') 
    g = sns.FacetGrid(df, row="classifier", col='metric', row_order=['Decision Tree', 'Logistic Regression', 'MLP'], col_order=features, height=3, aspect=1.25, margin_titles=True, despine=False, sharey=True)
    g.map_dataframe(sns.boxplot, x='algorithm', y='value', hue='algorithm', palette='colorblind', order=['-', 'FAWOS', 'FOS', 'Fair-RBH', 'Fair-RBU', 'HFOS'], showmeans=True, meanprops={"marker": "^", "markerfacecolor": "black", 'markeredgecolor': 'black'}, hue_order=['-','Fair-RBH', 'Fair-RBU', 'FAWOS', 'FOS', 'HFOS'])
    plt.tight_layout()
    plt.savefig(f'{save_path}/{dataset_name}_{bin}_results.pdf')
    plt.show()



for dataset_name, dfs_f, dfs_p in zip(dataset_names, dfs_fairness, dfs_performance):
    print(dataset_name)
    
    aggregate_runs_no_mean(dfs_f, dfs_p, dataset_name, 'bin')

In [ ]:
def construct_big_df(dfs, dataset_names, bin):
    dfs_new = []
    if bin:
        dict_ratios = {'_balanced_g': '1:1', 'mildly_imbalanced_g': '7:3', 'strongly_imbalanced_g': '9:1', 
                       '_balanced_c': '1:1', 'mildly_imbalanced_c': '7:3', 'strongly_imbalanced_c': '9:1'}
    else:
        dict_ratios = {'_balanced_g': '1:1:1:1', 't_mildly_imbalanced_g': '7:7:3:3', 't_strongly_imbalanced_g': '9:9:1:1', 't_strongly_mildly_imbalanced_g': '9:7:3:1',
                       '_balanced_c': '1:1', 'g_mildly_imbalanced_c': '7:3', 'g_strongly_imbalanced_c': '9:1', }
    for df, d_name in zip(dfs, dataset_names):
        g_imbalance = [dict_ratios[i] for i in dict_ratios.keys() if i in d_name and '_g' in i][0]
        c_imbalance = [dict_ratios[i] for i in dict_ratios.keys() if i in d_name and '_c' in i][0]
        print(d_name, g_imbalance, c_imbalance)
        df_whole = pd.concat(df).reset_index(drop=True)
        df_whole['group ratio'] = [g_imbalance] * len(df_whole)
        df_whole['class ratio'] = [c_imbalance] * len(df_whole)
        dfs_new.append(df_whole)
    big_df = pd.concat(dfs_new).reset_index(drop=True)
    return big_df

def aggregate_runs_no_mean_adult(df, column_to_use, classifier, bin, save_path='../figures'):
    
    df_num_cols = [c for c in df.columns if c not in ['classifier', 'algorithm', 'group ratio', 'class ratio']]
    df.loc[:, df_num_cols] = df.loc[:, df_num_cols].abs()
    df['classifier'].replace({'decision_tree': 'Decision Tree', 'logistic_regression': 'Logistic Regression', 'mlp': 'MLP'}, inplace=True)
    df['algorithm'].replace({'fair_rbu': 'Fair-RBU', 'fair_rbh': 'Fair-RBH', 'fos': 'FOS', 'fawos': 'FAWOS', 'hfos': 'HFOS'}, inplace=True)
    df = df.loc[df['classifier'] == classifier, :]
    df.drop(columns=['classifier'], inplace=True)
    try:
        df.rename(columns={'statistical_parity': 'SPD', 'average_absolute_odds': 'AAO', 'equal_opportunity': 'EOD'}, inplace=True)
    except KeyError:
        pass
    try:
        df.rename(columns={'accuracy': 'Accuracy', 'f1': 'F1', 'gmean': 'G-mean'}, inplace=True)
    except KeyError:
        pass
    features = [column_to_use]
    df_drop_columns = [c for c in df.columns if c not in ['classifier', 'algorithm', 'group ratio', 'class ratio', 'iteration', column_to_use]]
    df.drop(columns=df_drop_columns, inplace=True)
    #df = df.melt(id_vars=['group_imbalance', 'class_imbalance', 'algorithm'], value_vars=features)
    #df.rename(columns={'variable': column_to_use}, inplace=True)
    sns.set(font_scale=0.75, style='whitegrid') 
    g = sns.FacetGrid(df, row="class ratio", col='group ratio', row_order=['1:1', '7:3', '9:1'], col_order=['1:1', '7:3', '9:1'], height=3, aspect=1.25, margin_titles=True, despine=False)
    g.map_dataframe(sns.boxplot, x='algorithm', y=column_to_use, hue='algorithm', palette='colorblind', order=['-', 'FAWOS', 'FOS', 'Fair-RBH', 'Fair-RBU', 'HFOS'], showmeans=True, meanprops={"marker": "^", "markerfacecolor": "black", 'markeredgecolor': 'black'}, hue_order=['-','Fair-RBH', 'Fair-RBU', 'FAWOS', 'FOS', 'HFOS'])
    plt.tight_layout()
    plt.savefig(f'{save_path}/adult_imbalances_{bin}_results_{column_to_use}_{classifier}.pdf')
    plt.show()
    
def aggregate_runs_no_mean_adult_multi(df, column_to_use, classifier, bin, save_path='../figures'):
    
    df_num_cols = [c for c in df.columns if c not in ['classifier', 'algorithm', 'group ratio', 'class ratio']]
    df.loc[:, df_num_cols] = df.loc[:, df_num_cols].abs()
    df['classifier'].replace({'decision_tree': 'Decision Tree', 'logistic_regression': 'Logistic Regression', 'mlp': 'MLP'}, inplace=True)
    df['algorithm'].replace({'fair_rbu': 'Fair-RBU', 'fair_rbh': 'Fair-RBH', 'fos': 'FOS', 'fawos': 'FAWOS', 'hfos': 'HFOS'}, inplace=True)
    df = df.loc[df['classifier'] == classifier, :]
    df.drop(columns=['classifier'], inplace=True)
    try:
        df.rename(columns={'statistical_parity': 'SPD', 'average_absolute_odds': 'AAO', 'equal_opportunity': 'EOD'}, inplace=True)
    except KeyError:
        pass
    try:
        df.rename(columns={'accuracy': 'Accuracy', 'f1': 'F1', 'gmean': 'G-mean'}, inplace=True)
    except KeyError:
        pass
    features = [column_to_use]
    df_drop_columns = [c for c in df.columns if c not in ['classifier', 'algorithm', 'group ratio', 'class ratio', 'iteration', column_to_use]]
    df.drop(columns=df_drop_columns, inplace=True)
    #df = df.melt(id_vars=['group_imbalance', 'class_imbalance', 'algorithm'], value_vars=features)
    #df.rename(columns={'variable': column_to_use}, inplace=True)
    sns.set(font_scale=0.75, style='whitegrid') 
    print(df)
    g = sns.FacetGrid(df, row="class ratio", col='group ratio', row_order=['1:1', '7:3', '9:1'], col_order=['7:7:3:3', '9:7:3:1', '9:9:1:1'], height=3, aspect=1.25, margin_titles=True, despine=False)
    g.map_dataframe(sns.boxplot, x='algorithm', y=column_to_use, hue='algorithm', palette='colorblind', order=['-', 'FAWOS', 'Fair-RBH', 'Fair-RBU', 'HFOS'], showmeans=True, meanprops={"marker": "^", "markerfacecolor": "black", 'markeredgecolor': 'black'}, hue_order=['-','Fair-RBH', 'Fair-RBU', 'FAWOS', 'FOS', 'HFOS'])
    plt.tight_layout()
    plt.savefig(f'{save_path}/adult_imbalances_{bin}_results_{column_to_use}_{classifier}.pdf')
    plt.show()

whole_dataset = construct_big_df(dfs_performance, dataset_names, False)
for cls in ['Decision Tree', 'Logistic Regression', 'MLP']:
    aggregate_runs_no_mean_adult_multi(whole_dataset, 'G-mean', cls, False)